# Practical Python Design Patterns

Book: <u>Practical Python Design Patterns - Pythonic Solutions to Common Problems</u> _W. Badenhorst_

- [3. Factory Pattern](#factory)
- [4. Builder Pattern](#builder)
- [5. Adapter Pattern](#adapter)
- [6. Decorator Pattern](#decorator)
- [7. Facade Pattern](#facade)
- [8. Proxy Pattern](#proxy)
- [9. Chain of Responsibility](#chain)
- [10. Command Pattern](#command)
- [11. Interpreter Pattern](#interpreter)
- [12. Iterator Pattern](#iterator)
- [13. Observer Pattern](#observer)
- [14. State Pattern](#state)
- [15. Strategy Pattern](#strategy)
- [16. Template Method Pattern](#template)
- [17. Visitor Pattern](#visitor)
- [18. Model-View-Controller Pattern](#mvc)
- [19. Publish-Subscribe Pattern](#ps)



# <a class="anchor" id="builder">4. Builder Pattern</a>

The builder pattern does not use nmerous constructors; it uses a builder object (differently from an anti-pattern). This builder object receives each initialization parameter step by step and then returns the resulting constructed object as a single result.
the `builder pattern` separates the construction of an object from the way the object is represented, so the representation of the object can be changed without changing the process by which it is constructed.

**Anti-pattern** is the opposite of a software **pattern**. They are usually a general way to work around or attempt to solve a specific type of problem, but are also the wrong way to solve the problem. Considering that **The Goal** is to write clean code that is easy to debug, easy to update, and easy to extend, anti-patterns lead to code that is exactly the opposite.

In the `Builder Pattern` the two main players are:
- `Builder` : abstract class that knows how to build all the components of the final object.
- `Director`: controls the process of the building. Its output is a fully initialized object.

There is an instance (or instances) of `Builder` that the `Director` uses to build the objects.

**Builder pattern** focuses on building something step by step. For this, it is different from the **abstract factory** where a family of products can be created and returned immediatly using polimorphism. **Builder pattern** separates the construction of a complex object from its representation. 

A furthe advantage of this separation is the reduction of object size, which results in cleaner code. There is better control over the construction process and themodular nature allow us to easily make changes to the internal representation of objects.

The biggest downside is that a `ConcreteBuilder` must be created for each type of product to be created.

In [ ]:
from abc import ABCMeta, abstractmethod

class Director(object, metaclass=ABCMeta):
    
    def __init__(self):
        self._builder = None
    
    @abstractmethod
    def construct(self):
        pass
    
    def get_constructed_object(self):
        return self._builder.constructed_object

class Builder(object, metaclass=ABCMeta):
    def __init__(self, constructed_object):
        self.constructed_object = constructed_object
        
class Product(object):
    def __init__(self):
        pass
    
    def __repr__(self):
        pass
    
class ConcreteBuilder(Builder):
    "The resulting object is able to construct objects."
    pass

class ConcreteDirector(Director):
    pass

## Exercises


# <a class="anchor" id="adapter">5. Adapter Pattern</a>

When dealing with something that was previously implemented (even by someone else), an adapter is needed to fit "what is available" to "what is needed". The pattern is useful to make things work after they are designed. Below the main elements of the `adapter pattern`:
- Target: defines the domain-specific interface the client uses
- Client: uses objects that conform to the _Target_ interface
- Adaptee: the interface to be altered because the object does not conform to the _Target_
- Adapter: the code that changes what there is in the _Adaptee_ to what is wanted in the _Client_

Adapters occurs at all levels of complexity. In Python, the concept of adapters extends beyond classes and their instances. Often, callables are adapted via decorators, closures, and functools.

For every new interface to be adapted, an `AdapterObject` class is needed: such class is constructed with two arguments in its constructor, one of which is the instance of the adapteee. Each of the adapters also needs to implement the needed function.

If the parameters passed to each `provided_functin` remain the same as those of the `required_function`, there can be created a more generic adapter that no longer needs to know anything about the adaptee: only an object and the `provided_function` must be supplied. 

Here below the generic implementation of the idea:

In [ ]:
class ObjectAdapter(object):
    def __init__(self, what_i_have, provided_function):
        self.what_i_have = what_i_have
        self.required_function = provided_function
    
    def __getattr__(self, attr):
        return getattr(self.what_i_have, attr)

## Sample Problem

The first solution that comes to mind is simply changing what is needed to fit what is available. When dealing with few interfaces, it is not a bad solution. When there are too many interfaces, too many `if` statements would be needed and it's not the best solution to manage.

In [ ]:
class WhatIHave(object):
    def provided_function_1(self): pass
    def provided_function_2(self): pass

class WhatIWant(object):
    def required_function(self): pass

class Client(object):
    def __init__(some_object):
        self.sme_object = some_object
    
    def do_something(self):
        if self.some_object.__class__ == WhatIHave:
            self.some_object.provided_function_1()
            self.some_object.provided_function_2()
        else if self.some_object.__class__ == WhatIWant:
            self.some_object.required_function()
        else:
            print("Class of self.some_object not recognized.")

## Class Adapter

Another way to solve this problem is to create several interfaces that use polymorphism to inherit both the expected and the provided interfaces. So, the target interface can be created as pure interface class.

In [ ]:
class MyAdapter(WhatIHave, WhatIWant):
    def __init__(self, what_i_have):
        self.what_i_have = what_i_have
    
    def provided_function_1(self):
        self.what_i_have.provided_function_1
    
    def provided_function_2(self):
        self.what_i_have.provided_function_2
        
    def required_function(self):
        self.provided_function_1()
        self.provided_function_2()

class ClientObject():
    def __init__(self, what_i_want):
        self.what_i_want = what_i_want
    
    def do_something(self):
        self.what_i_want.required_function()

if __name__ == '__main__':
    adaptee = WhatIHave()
    adapter = MyAdapter(adaptee)
    client = ClientObject(adapter)
    
    client.do_something()

## Object Adapter Pattern

Make use of composition instead of inheritance. An adapter could then contain the class it wraps and make calls to the instance of the wrapped object. This approach reduces the implementation complexity. The pattern is useful to make things work after they are designed. It is a way of providing a different interface to sujects to which the interface are provided from the interface provided.

The adapter below inherits from `IterfaceSuperClass` and takes an instance as a parameter in its constructor.
It implements a `required_function` method, which returns the result of calling the `provided_function_1()` method of its wrapped `WhatIHave` object. all other calls on the class are passed to its `what_i_have` instance via the `__getattr__()` method. when the Python interpreter does an attribute lookup on the object but can't find it, the `__getattr__()` method is called, and the attribute lookup is passed tot he `self.what_i_have` object.

In [ ]:
class ObjectAdapter(InterfaceSuperClass):
    def __init__(self, what_i_have):
        self.what_i_have = what_i_have
        
    def required_fuction(self):
        return self.what_i_have.provided_function_1()
    
    def __getattr__(self, attr):
        # everything else is handled by the wrapped object
        return getattr(self.what_i_have, attr)

### Duck Typing

In python we only care about whether an object offers the interface elements we need. If it does, we can use it like an instance of the interface without the need to declare a common parent (here below the parent class is the default `object` instead of another class previously defined). The pattern **uses composition rather than inheritance**. This respects the `open/closed` principle (by _B. Meyer_):

_software entities (classes, modules, functions, etc.) should be open for extension, but closed for modification._

In [ ]:
class ObjectAdapter(object):
    def __init__(self, what_i_have):
        self.what_i_have = what_i_have
        
    def required_fuction(self):
        return self.what_i_have.provided_function_1()
    
    def __getattr__(self, attr):
        # everything else is handled by the wrapped object
        return getattr(self.what_i_have, attr)

## Exercises


# <a class="anchor" id="decorator">6. Decorator Pattern</a>

To decorate a function, we need to return an object that can be used as a function.
In Python everything is an object: functions are objects with a special `__call()__` method. If the decorator returns an object with a `__call()__` method, the result can be used as a function.
In the `Decorator Pattern` the interface does not change, but new functionality is somehow adeed.

<u>Example of **ProfilingDecorator** class</u>

```python
import time

class ProfilingDecorator(object):
    def __init__(self, f):
        self.f
       
    def __call__(self, *args):
        start_time = time.time()
        result = self.f(*args)
        end_time = time.time()
        print("[Time elapsed for running function] {}".format(end_time-start_time))
        
        return result
  
@ProfilingDecorator
def someFunc(*args):
    pass
```
The decorated function is saved as an attribute of the object during initialization.

The decorator is a _unary funciton_ (i.e. a function that takes a single argument) that takes a function to be decorated as its argument.

All the code that interacts with the decorated function can remain the same as when the funciton was undecorated.

<u>Example of **profiling_decorator** function</u>

```python
def profiling_decorator(f):
    def wrapped_f(*args):
        start_time = time.time()
        result = self.f(*args)
        end_time = time.time()
        print("[Time elapsed for running function] {}".format(end_time-start_time))
        
        return result
    return wrapped_f

@profiling_decorator
def someFunc(*args):
    pass           
```

The decorator function must return a function to be used when the decorated function is called. The returned function is used to wrap the decorated function.

**Closure**: to have a closure, you have to have one function (i.e. `profiling_decorator`) returning another function (i.e. `wrapped_f`) nested within itself, with the nested function referring to a variable (i.e. `f`) within the scope of the enclosing function.

## Decorating Classes

Somethimes it would be useful to decorate the class and have the Python know to apply the decoration to each method in the class. This can be achieved through a class and the `__getattribute__()` magic metho: it is used to retrieve methods and attributes for an object, and by overriding this method it is possible to add the decorator.

In [ ]:
import time
from functools import wraps

def profiling_wrapper(f):
    @wraps(f)
    def wrap_f(*args, **kwargs):
        start_time = time.time()
        result = f(*args, **kwargs)
        end_time = time.time()
        elapsed_time = end_time - start_time
        print(f"time elapsed {elapsed_time}")
        
        return result
    
    return wrap_f

def profile_all_class_methods(Cls):
    class ProfiledClass(object):
        def __init__(self, *args, **kwargs):
            self.inst = Cls(*args, **kwargs)
        
        def __getattribute__(self, s):
            try:
                x = super(ProfiledClass, self).__getattribute__(s)
            except AttributeError:
                pass
            else:
                x = self.inst.__getattribute__(s)
                if type(x) == type(self.__init__):
                    return profiling_wrapper(x)
                else:
                    return x
    return ProfiledClass

@profile_all_class_methods
class DoSomeStuff(object):
    def someFunc(self):
        pass                

## Exercises

# <a class="anchor" id="facade">7. Facade Pattern</a>

System eolve, and as they grow they can become very complex. To keep the whole system simple, all the functionalities must be hidden from the client.

The `Facade Pattern` is specifically used to create a complexity-limiting interface to a sub-system or collection of sub-systems. It is used not just to encapsulate a single object but also to provide a wrapper that presents a simplified interface to a group of complex sub-systems that is easy to use without unneeded functions or complexity.

The facade should decide which internal classes and representations to use. It is also within the facade that you will make changes in terms of functions to be assigned to eternal or updated internal systems when the time comes to do so.

The simplified interface remains unaware of the complexity under the hood without reducing the power inherent in the sub-system.

In [ ]:
class Facade(object):
    def __init__(self):
        pass
    
    @staticmethod
    def doSomething(*args, **kwargs):
        return ClassDoSomething(*args, **kwargs)
    
    @staticmethod
    def doMore(*args, **kwargs):
        return ClassDoMore(*args, **kwargs)

## Exercises

# <a class="anchor" id="proxy">8. Proxy Pattern</a>

A `Proxy Pattern` provides the same interface as the original object, but it controls access to the original object. It can be applied when there are functions that are called very often.

<u>Memoization:</u> the act of saving the result of a function call for later use. Whenever there is a function being called multiple times, with the value repeated, it would be useful to store the response of the calculation in order to avoid the process of calculating the value again.

```python
def fib_cached(n, cache):
    if n < 2:
        return 1
    
    if n in cache:
        return cache[n]
    
    cache[n] = fib_cached(n-2, cache) + fib(n-1, cache)
    
    return cache[n]

n = 100
fib_sequence = [fib_cached(x, cache) for x in range(0, n)]
```

## Proxy Pattern

A `proxy` provides the same interface as the original object, but it controls access to the original object. As part of this control, it can perform other tasks before and after the original object is accessed. It tipucally has three parts:
- _client_: it requires access to some object.
- _object_: its access is requested by the _client_.
- _proxy_: it controls the access to the _object_.

The ideal situation is to have a class that functions as an interface to the calculator class. The client should not be aware of this class, in that the client only codes toward the interface of the original class, with the proxy providing the same functionality and results as the original class.

`Proxy` types:
- `Remote Proxy`: to abstract the location of an object. It appears to be a local resource to the client.
- `Virtual Proxy`: to delay object creation. The target object can be created once needed.
- `Prtection Proxy`: to restrict access to information and methods on the target object.

With the `Proxy Pattern` the interface remains constant, with some actions taking place in the background. Conversely, the `Adapter Pattern` is targeted at changing the interface.

In [2]:
import time

class RawCalculator(object):
    def fib(self, n):
        if n < 2: 
            return 1
        return self.fib(n-2) + self.fib(n-1)

def memoize(fn):
    """Memizing function, it works with any function passed to it."""
    __cache = {}
    def memoized(*args):
        key = (fn.__name__, args)
        if key in __cache: 
            return __cache[key]
        __cache[key] = fn(*args)
        return __cache[key]
    return memoized

class CalculatorProxy(object):
    def __init__(self, target):
        self.target = target
        
        fib = getattr(self.target, 'fib')
        setattr(self.target, 'fib', memoize(fib))  # overriding of fib method
    
    def __getattr__(self, name):
        return getattr(self.target, name)
    
if __name__ == "__main__":
    calculator = CalculatorProxy(RawCalculator())
    start_time = time.time()
    fib_sequence = [calculator.fib(x) for x in range(80)]
    end_time = time.time()
    print("Calculating the list of {} Fibonacci numbers took {} seconds".
          format(len(fib_sequence), end_time - start_time)
         )

Calculating the list of 80 Fibonacci numbers took 0.0 seconds


## Exercises

# <a class="anchor" id="chain">9. Chain of Responsibility</a>

The `Chain and Responsibility` pattern allows us to shuffle and alter the handlers used at runtime. It can serve to encapsulate a processing of elements into a pipeline. A good rule of thumb is to use it wherever you have more than one potential handler for a request, and thus do not know beforehand which handler or handlers would best handle the requests you will receive.

_The Middleware Layer is a key component of web frameworks: it sits in the middle between the client making requests of the app and the actual application. Both the requests from the client and the responses from the main application pass through it. Also the **routing** mechanism is part of the middleware layer._

About web frameworks:
- simple `Request` object (sent to the server):

```python
class Request(object):
    def __init__(self, headers, url, body, GET, POST):
        self.headers = headers
        self.url = url
        self.body = body
        self.GET = GET
        self.POST = POST
```

- simple `Response` object (returned to the client):

```python
class Response(object):
    def __init__(self, headers, status_code, body):
        self.headers = headers
        self.status_code = status_code
        self.body = body
```

## Chain Responsibility Principle

**Chain Responsibility Principle**: every piece of code does one thing and only one thing.

Below the basic concept:

```python
def function_1():
    print('function_1')

def function_2():
    print('function_2')

def function_3():
    print('function_3')

def main_function():
    function_1()
    function_2()
    function_3()

if __name__ == '__main__':
    main_function()
```

Since it is not ideal to have the `main_function` call each of the functions in order (because it leads to a messy code), a better implementation leads to create a way to make a single call and then have the functions called dynamically. In this way, each function is clearly separated int its wn unit of code that can be plugged into, or removed from, the chain of classes. Each handler cares only abuot its own execution and ignores what happens when another handler executes because of the query.

In [6]:
class CatchAll(object):
    def __init__(self):
        self.next_to_execute = None
    def execute(self):
        print('end reached.')
        
class Function1Class(object):
    def __init__(self):
        self.next_to_execute = CatchAll()
    def execute(self):
        print('function_1')
        self.next_to_execute.execute()
        
class Function2Class(object):
    def __init__(self):
        self.next_to_execute = CatchAll()
    def execute(self):
        print('function_2')
        self.next_to_execute.execute()
        
class Function3Class(object):
    def __init__(self):
        self.next_to_execute = CatchAll()
    def execute(self):
        print('function_3')
        self.next_to_execute.execute()
        
class Function4Class(object):
    def __init__(self):
        self.next_to_execute = CatchAll()
    def execute(self):
        print('function_4')
        self.next_to_execute.execute()

def main_function(head):
    head.execute()
    
if __name__ == '__main__':
    hd = Function1Class()
    
    current = hd
    current.next_to_execute = Function2Class()
    
    current = current.next_to_execute
    current.next_to_execute = Function3Class()
    
    current = current.next_to_execute
    current.next_to_execute = Function4Class()
    
    main_function(hd)

function_1
function_2
function_3
function_4
end reached.


## Exercises


# <a class="anchor" id="command">10. Command Pattern</a>

The `Command Pattern` decouples the request for execution from the actual execution.

Used to send an instruction or set of instructions from one object to another, while keeping these object loosely coupled. It follows that everything needed to execute the instruction should be encapsulated in some kind of data structure. the _client_ that initiates the execution does nt have to know anything about way in which the instruction will be executed. the target object is called _receiver_, it is an instance of a class that can execute the method given the encapsulated information. All of this relies in an bject called an _invoker_ that decides when the method on the receiver will execute.

It is fundamental that the command pattern isolates the invoker from the receiver. It also separates the time the execution is set up from the time it is procesed.

Commands to be executed can be queued even if the receiver is busy executing another command. Having all the information in an object in the queue allows the system to deal with all incoming commands withut losing important commands while executing some other command. **New behaviors are dynamically created at runtime!**

Sample `Command Pattern`:

In [ ]:
class Command(object):
    def __init__(self, receiver, text):
        self.receiver = receiver
        self.text = text
    def execute(self):
        self.receiver.print_message(self.text)
        
class Receiver(object):
    def print_message(self, text)
    print("Message received: {}".format(text))
    
class Invoker(object):
    def __init__(self):
        self.commands = []
    def add_command(self, command):
        self.commands.append(command)
    def run(self):
        for command in self.commands:
            command.execute()

if __name__ == '__main__':
    receiver = Receiver()
    command1 = Command(receiver, "Execute command 1")
    command2 = Command(receiver, "Execute command 2")
    
    invoker = Invoker()
    invoker.add_command(command1)
    invoker.add_command(command2)
    invoker.run()

The `Command Pattern` can also build a multi-level undo stack:
```python
class Invoker(object):
    def __init__(self):
        self.commands = []
        self.undo_stack = []
        
    def add_new_command(self, command):
        self.commands.append(command)
    
    def run(self):
        for command in self.commands:
            command.execute()  # execution function of command (e.g. - for Subtraction)
            self.undo_stack.append(command)
    
    def undo(self):
        undo_command = self.undo_stack.pop()
        undo_command.undo()   # undo function of command (e.g. + for Subtraction)
        
# sample command class

class SubtractCommand(object):
    def __init__(self, receiver, value):
        self.receiver = receiver
        self.value = value

    def execute(self):
        self.receiver.subtract(self.value)
        
    def undo(self):
        self.receiver.add(self.value)

# receiver         

class Accumulator(object):
    def __init__(self, value):
        self.value = value
    
    def add(self, value):
        self.value += value
    
    def subtract(self, value):
        self.value -= value
```

The heart of this design pattern is the translation of method calls into data that can be saved in a variable, passed to a method or function as a parameter, and returned from a function as a result. The result of applying this pattern is that functions or methods become **first-class citizens**. When functions are first class citizens variables can point to functions, functions can be passed as a parameter to other functions, and they can be returned as the result from executing a function.

All functions get passed all that is needed for execution, with no global state, and functions are only executed when an actual result needs to be returned.

## Examples

# <a class="anchor" id="interpreter">11. Interpreter Pattern</a>

**General-Purpose Languages**: designed to be used to solve any problem.
**Domain-Specific Languages (DSL)**: designed to do only one thing, but extremely well. These languages are helpful for people who are experts in some specific domain but are not programmers. Two types:
- _external_: have external code written in an external file or a string. The string or file is the read and parsed by the application before it gets used (e.g. CSS). A parser is needed (e.g. `PyParsing`).
- _internal_: use features of the language (e.g. Python with `aloe` module) to enable people writing code that resembles the domain syntax.

When developing DSL here are the main steps:
1. understand your domain
2. model your domain
3. implement your domain

The idea is to create a syntax that is significantly less comprehensive than a general-purpose language, yet significantly more expressive, specifically as it relates to the domain in question.
Two tasks to be accomplished:
1. define the language, i.e. the semantics (meaning) and syntax (structure):
    - identify things
    - identify actions
2. write code that will be able to take the language as input and translate it into a form that a machine can understand and action.
    - generalize the elements
   
## Composite Pattern

For a container with elements that could be containers themeselves. The `Composite Pattern` defines both composite (i.e. non-terminals) classes and leaf (i.e. terminals) classes that can be used to construct a composite component, such as a special rule.

```python
class Leaf(object):
    def __init__(self, *args, **kwargs):
        pass
    def component_function(self):
        print('Leaf')

class Composite(object):
    def __init__(self, *args, **kwargs):
        self.children = []
    def add(self, child):
        self.children.append(child)
    def remove(self, child):
        self.children.remove(child)
```

## Interpreter Pattern

It fits the need of people who are willing to tinker and tweak tomake the software fit their needs more perfectly.

Every expression type gets a class, and every class has an `interpret` method. A class and an object to store the global context will also be needed. This context is passed to the `interpret` function of the next object in the interpretation stream (assume that parsing already took place). The interpreter recursively tracerses the container object until the answer to the problem is reached.

```python
class NonTerminal(object):
    def __init__(self, expression):
        self.expression = expression
    def interpret(self):
        self.expression.interpret()
        
class Terminal(object):
    def interpret(self):        
        pass
```    


## Exercises

# <a class="anchor" id="iterator">12. Iterator Pattern</a>

The `Iterator Pattern` creates an object that can use an inetrface that a collection data type can inherit, which would allow it to generalize the action of traversing the contents of the collection.
The `Container` class implements the iterator protocol.

In [9]:
import abc

class Iterator(metaclass=abc.ABCMeta):
    @abc.abstractmethod
    def has_next(self): pass
    
    @abc.abstractmethod
    def next(self): pass

class Container(metaclass=abc.ABCMeta):
    @abc.abstractmethod
    def getIterator(self): pass

class MyListIterator(Iterator):
    def __init__(self, my_list):
        self.index = 0
        self.list = my_list.list
        
    def has_next(self):
        return self.index < len(self.list)
    
    def next(self):
        self.index += 1
        return self.list[self.index - 1]

class MyList(Container):
    def __init__(self, *args):
        self.list = list(args)
    def getIterator(self):
        return MyListIterator(self)
    
if __name__ == '__main__':
    my_list = MyList(1, 2, 3, 4, 5, 6)
    my_iterator = my_list.getIterator()
    
    while my_iterator.has_next():
        print(my_iterator.next())

1
2
3
4
5
6


## Exercises

# <a class="anchor" id="observer">13. Observer Pattern</a>

Used whenever there is the need to dynamically decouple the source of a call from the called code. As a rule, whenever there s a publish-subscribe relationship between a single object (i.e. the `Observable`) and a set of observers, it is a good candidate for `Design Pattern`. Whenever the _Observable_ object changes its states and is polled for a chaneg, it alerts all the _Observers_ registered with it that they need to activate a callback. 
Although the `Observer Pattern` allows decoupling the objects that are observed frm knowing anything about the object oobserving them, the observer objects stll need to know which objects they need to observe. There is still a bt of coupling. Such pattern works well whena number of potential observable classes that an oobserver needs to register with is limited.
The level of coupling between objects is the degree of knowledge that one object needs with regard to other objects that it interacts with. The more loosely objects are cupled, the less knowledge they have about each other, and the more flexible the object-orieted system is. Loosely coupled systems have fewer interdependencies between objects and as such are easier to update and maintain.

In the `Observer Pattern` there are two types of objects:
- `Observable` class: can be watched by other classes.
- `Observer` class: will be alerted whenever the two classes undergoes a change.

`Observer Pattern` definition: _A software design pattern in which an object, called the subject, maintans a list of its dependents, called observers, and notfies the automatically of any state changes, usually by calling one of their methods. It is mainly used to implement distributed event handling systems._

The problems that the `Observer Pattern` solves are those where a group of objects has to respond to the change f state in some other object and d so withut causing more coupling inside the system. It s concerned with the management of events r responding to change of state in some sort of network of objects.

In [6]:
class ConcreteObserver(object):
    def update(self, observed):
        print("Observing: {}".format(observed))
        
class Observable(object):
    def __init__(self):
        self.callbacks = set()
    def register(self, observer):
        self.callbacks.add(observer)
    def unregister(self, observer):
        self.callbacks.discard(observer)
    def unregister_all(self):
        self.callbacks = set()
    def update_all(self):
        for callback in self.callbacks:
            callback.update(self)

def main():
    observed = Observable()
    observer1 = ConcreteObserver()
    
    # observed.register(lambda x: observer1.update(x))   # in pdf, this does not work
    observed.register(observer1)
    observed.update_all()

if __name__ == '__main__':
    main()            

Observing: <__main__.Observable object at 0x000002B550C8F088>


## Exercises

# <a class="anchor" id="state">14. State Pattern</a>

**State Diagram**: a graph where nodes represent the state of the system and edges are transitions between one node in the system and another.

A naive way to translate the daigram into runnable code is to create an object that represents the state machine. The object will have an attribute for its state, which will determine hw it reacts to input.
All object-oriented systems concern themselves with the actors in a system and how the actions of each impact the other actors and the system as a whole. This is why **state machine** is so helpful in modeling the state of an object and the things that cause said object to react.
The `State Pattern` is used to encapsulate behavior variations based on the internal state of an object.

All state machines are composed of states and the transitions taking the machine from one state to another based on certain inputs. Usually, the state machine will also execute some actions while in a state before transitioning to another state.

In [7]:
class State(object):
    pass

class ConcreteState1(State):
    def __init__(self, state_machine):
        self.state_machine = state_machine
    def switch_state(self):
        self.state_machine.state = self.state_machine.state2
        
class ConcreteState2(State):
    def __init__(self, state_machine):
        self.state_machine = state_machine
    def switch_state(self):
        self.state_machine.state = self.state_machine.state1

class StateMachine(object):
    def __init__(self):
        self.state1 = ConcreteState1(self)
        self.state2 = ConcreteState2(self)
        self.state = self.state1  # initialized to state1
        
    def switch(self):
        self.state.switch_state()
        
    def __str__(self):
        return str(self.state)

def main():
    state_machine = StateMachine()
    print(state_machine)
    
    state_machine.switch()
    print(state_machine)
    
if __name__ == '__main__':
    main()

What should be tested about a state machine:
1. The state machine initializes correctly;
2. The action method for each concrete `State` class does what it should do (e.g. return the correct value).
3. For a give input, the machine transitins to the correct subsequent state.

Some unit testing with `unittest` (library framework):

In [13]:
import unittest

class GenericStatePatternTest(unittest.TestCase):
    def setUp(self):
        self.state_machine = StateMachine()
    
    def tearDown(self):
        pass
    
    def test_state_machine_initializes_correctly(self):
        sefl.assertInstance(self.state_machine.state, ConcreteState1)
    
    def test_switch_from_state_1_to_state_2(self):
        self.state_machine.switch()
        self.assertIsInstance(self.state_machine.state, ConcreteState2)
        
    def test_switch_from_state2_to_state1(self):
        self.state_machine.switch()
        self.state_machine.switch()
        
        self.assertIsInstance(self.state_machine.state, ConcreteState1)
        
if __name__ == '__main__':
    unittest.main()

E
ERROR: C:\Users\i (unittest.loader._FailedTest)
----------------------------------------------------------------------
AttributeError: module '__main__' has no attribute 'C:\Users\i'

----------------------------------------------------------------------
Ran 1 test in 0.001s

FAILED (errors=1)


SystemExit: True

c:\users\i.stiaccini\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3449: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## Exercises

# <a class="anchor" id="strategy">15. Strategy Pattern</a>

To switch between different ways of solving a problem, being able to pick a strategy at runtime and then run with it. allthis, by applying a modular solution that would allow passing in new strategies on the fly.

The `Strategy Pattern` allows writing code that uses some strategy, to be selected at runtime, without knowing anything about the strategy other that it follows some execution signature. It is composed by an `Executor` to which the strategies are passed, and then executed.

In [14]:
class StrategyExecutor(object):
    def __init__(self, strategy=None):
        self.strategy = strategy
    def execute(self, *args):
        if self.strategy is None:
            print("Strategy not implemented...")
        else:
            self.strategy.execute(*args)

class Strategy1(object):
    def execute(self, *args):
        for a in args:
            print('Strategy1:', a)

class Strategy2(object):
    def execute(self, *args):
        for a in args:
            print('Strategy2:', a)
            
if __name__ == '__main__':
    no_strategy = StrategyExecutor()
    strategy_1 = StrategyExecutor(Strategy1())
    strategy_2 = StrategyExecutor(Strategy2())
    no_strategy.execute(1, 2, 3)
    strategy_1.execute(4, 5, 6)
    strategy_2.execute(7, 8, 9, 0)

Strategy not implemented...
Strategy1: 4
Strategy1: 5
Strategy1: 6
Strategy2: 7
Strategy2: 8
Strategy2: 9
Strategy2: 0


A more **pythonic** version with functions:

In [18]:
def executor(*args, func=None):
    if func is None:
        print("Strategy not implemented...")
        return
    
    return func(*args)

def strategy1(*args):
    for a in args:
        print('strategy1:', a)
        
def strategy2(*args):
    for a in args:
        print('strategy2:', a)
        
if __name__ == '__main__':
    executor(1, 2)
    executor(3, 4, func=strategy1)
    executor(5, 6, 7, 8, func=strategy2)

Strategy not implemented...
strategy1: 3
strategy1: 4
strategy2: 5
strategy2: 6
strategy2: 7
strategy2: 8


# <a class="anchor" id="template"> 16. Template Method Pattern</a>

The `Template Method Pattern` provides a method template that can be followed to implement a specifc process step by step, and then that template can be used in many different scenarios by simply changing a couple of details.

Functions that clearly step in the same process should live in a single entity instead of being scattered.

In [ ]:
import abc

class TemplateAbstractBaseClass(metaclass = abc.ABCMeta):
    def template_method(self):
        self._step_1()
        self._step_2()
        self._step_n()
    
    @abc.abstractmethod
    def _step_1(self): pass

    @abc.abstractmethod
    def _step_2(self): pass

    @abc.abstractmethod
    def _step_n(self): pass

class ConcreteImplementationClass(TemplateAbstractBaseClass):
    def _step_1(self): pass
        
    def _step_2(self): pass

    def _step_n(self): pass

## Exercises

# <a class="anchor" id="visitor"> 17. Visitor Pattern</a>

Tease apart a complex piece of functionality into more discrete parts, then abstract these parts in such a way that the one does not need to be intimately familiar with the other.

In [ ]:
import abc

class Visitable(object):
    def accept(self, visitor):
        visitor.visit(self)
        
class CompositeVisitable(Visitable):
    def __init__(self, iterable):
        self.iterable = iterable
        
    def accept(self, visitor):
        for element in self.iterable:
            element.accept(visitor)
        visitor.visit(self)
        
class AbstractVisitor(object):
    __metaclass__ = abc.ABCMeta
    
    @abc.abstractmethod
    def visit(self, element):
        raise NotImplementedError("A visitor needs to define a visit method")
        
class ConcreteVisitable(Visitable):
    def __init__(self):
        pass
    
class ConcreteVisitor(AbstractVisitor):
    def visit(self, element):
        pass

## Exercises

# <a class="anchor" id="mvc"> 18. Model-View-Controller Pattern</a>

The `Model-View-Controller Pattern` is a pattern of patterns. To be applied when the program receives some form of a request that must be interpreted and then kick off some relevant actions. It is sub-divided into three main objects:
- **Controller** : all of the program processing happens inside an object that exists solely for controlling the flow of the program. This oobject handles actionsliker requesting data, receiving data, and dispatching responses to the command line. **Controllers** are the glue that holds the system together, and usually the place where the most action takes place. 
It is the hearth of the pattern, the part that marshalls all other classes. Users interact with the controller, and through it the whole system. It takes the following actions:
    - takes the user input;
    - handles all the business logic;
    - gets data from the model;
    - sends the data to the vew to be transformed into the representation to be returned to the user.
- **Model** : once a request is received and the controller decides what must happen to the request, some data from the system is needed. The structural representationo of data is referred to as a data model, and so the part of the program that deals with the data (getting, setting, updating, and deleting data) is called the **model**. It should be a program-side interface t data that abstracts away the need to directly interact with the data store, allowing to switch frma  file-based store to some key-value store or a full-on relational database system. Often, a model will contain the fields used as attrbutes on the object, allowing to interact with the database as with any other object.
- **View** : part of the code that deliceries the information back to the user. it should only deal with the output or rendering of data passed to it into some sort of format to be returned to the user.

**There is no business logic in Model neither in View!**




In [ ]:
class GenericContrller(object):
    def __init__(self):
        self.model = GenericModel()
        self.view = GenericView()
    
    def handle(self, request):
        data = self.model.get_data(request)
        self.view.generate_response(data)
        
class GenericModel(object):
    def __init__(self):
        pass
    
    def get_data(self, request):
        return {'request': request}
    
class GenericView(object):
    def __init__(self):
        pass
    
    def generate_response(self, data):
        print(data)
        

## Examples

# <a class="anchor" id="ps"> 19. Publish-Subscribe Pattern</a>

The `Publish-Subscribe Pattern` allows to fully decouple the observer and the observables. Neither the observer nor the observable need to know anything about each other. Each class and its instances should be able to change wthout any changes being needed on the other side of the equation:
- **publishers** : blind observables. Deals with publishing.
- **subscribers** : disconnected observers. Deals with processing.

The following classes will also be added to:
- **Message** : class that culd be published and processed.
- **Dispatcher** : the class thrugh which all the messages pass, it has the aim to provide a single location to which publishers send messages. The same location keeps an index of all the subscribers, as a result the number of publishers and subscribers can vary without having any impact on the rest of the system.

In [1]:
class Message(object):
    def __init__(self):
        self.payload = None
        self.topic = "all"
        
class Subscriber(object):
    def __init__(self, dispatcher, topic):
        dispatcher.subscribe(self, topic)
    def process(self, message):
        print("Message: {}".format(message.payload))
        
class Publisher(object):
    def __init__(self, dispatcher):
        self.dispatcher = dispatcher
    def publish(self, message):
        self.dispatcher.send(message)
        
class Dispatcher(object):
    def __init__(self):
        self.topic_subscribers = dict()
    def subscribe(self, subscriber, topic):
        self.topic_subscribers.setdefault(topic, set()).add(subscriber)
    def unsubscribe(self, subscriber, topic):
        self.topic_subscribers.setdefault(topic, set()).discard(subscriber)
    def unsubscribe_all(self, topic):
        self.subscribers = self.topic_subscribers[topic] = set()
    def send(self, message):
        for subscriber in self.topic_subscribers[message.topic]:
            subscriber.process(message)
            
def main():
    dispatcher = Dispatcher()
    publisher_1 = Publisher(dispatcher)
    subscriber_1 = Subscriber(dispatcher, 'topic1')
    message = Message()
    message.payload = "My Payload"
    message.topic = 'topic1'
    publisher_1.publish(message)
    
if __name__ == "__main__":
    main() 

Message: My Payload
